In [ ]:
#| default_exp signature_validators

In [ ]:
#| export
from pymoq.argument_validators import ArgumentValidator, ArgumentFunctionValidator
from typing import Any

In [ ]:
#| export
from fastcore.test import test_fail

## Signature validators
> Checking if signatures are valid

A signature validator is simply a collection of argument validators. Its `is_valid` methods checks for a given list of arguments if all argument validators return valid.

In [ ]:
#| export
class SignatureValidator:
    "This class holds a list of argument validators and can evaluate a list of arguments against those validators"
    def __init__(self, argument_validators: list[ArgumentValidator]):
        self.argument_validators = argument_validators
        self._named_validators = {validator.name: validator
                                  for validator in self.argument_validators}
        
        self._positional_validators = {validator.position: validator
                                      for validator in self.argument_validators}
        
        names = [validator.name for validator in self.argument_validators]
        if len(names) != len(set(names)):
            raise ValueError(f"List of argument validators contains duplicate names: {names}")
            
        positions = [validator.position for validator in self.argument_validators]
        if len(positions) != len(set(positions)):
            raise ValueError(f"List of argument validators contains duplicate positions: {positions}")
        
    def is_valid(self, *args: list[Any], **kwargs: dict[str, Any]) -> bool:
        if len(args) > len(self.argument_validators): return False
    
        # positional arguments
        for position, value in enumerate(args):
            if not position in self._positional_validators.keys(): return False
            
            if not self._positional_validators[position].is_valid(value): return False
        
        # named arguments
        for name,value in kwargs.items():
            if name not in self._named_validators: return False
        
            if not self._named_validators[name].is_valid(value): return False
        
        return True

In [ ]:
any_int = ArgumentFunctionValidator(lambda v: isinstance(v, int), "firstArgument", 0)
second_any_int = ArgumentFunctionValidator(lambda v: isinstance(v, int), "secondArgument", 1)

s = SignatureValidator([any_int, second_any_int])

Calling a signature validator with only positional arguments works as expected:

In [ ]:
assert s.is_valid(1,1)
assert not s.is_valid(1,"1")
assert not s.is_valid("1", 1)

Named arguments:

In [ ]:
assert s.is_valid(1, secondArgument=1)
assert not s.is_valid(1, named="1")

#### Edge cases

Argument validators contain more than one element with the same name

In [ ]:
test_fail(lambda : SignatureValidator([any_int, any_int]), "duplicate names")

Argument validators contain more than one element with the same position

In [ ]:
second_any_int._position = any_int._position
test_fail(lambda : SignatureValidator([any_int, second_any_int]), "duplicate positions")
second_any_int._position = any_int._position + 1

More arguments than validators:

In [ ]:
assert not s.is_valid(1,1,1)

# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()